# Exact Cover Problem

In mathematics, given a collection S of subsets of a set X, an exact cover is a subcollection S* of S such that each element in X is contained in exactly one subset in S*. One says that each element in X is covered by exactly one subset in S*. An exact cover is a kind of cover.

## Installation

Please first install blueqat

```bash
pip install blueqat
```

And prepare for libraries


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import blueqat.opt as wq

## QUBOmatrix

Now we have 2 cost function and constraint term.

$E_{A} = A \sum _ { \alpha = 1 } ^ { n } \left( 1 - \sum _ { i : \alpha \in V _ { i } } x _ { i } \right) ^ { 2 }$

$E_{A} = A \sum _ { \alpha = 1 } ^ { n } \{ 1 - 2\sum _ { i : \alpha \in V _ { i } } x _ { i } + ( \sum _ { i : \alpha \in V _ { i } } x _ { i } ) ^ { 2 } \} $

Ignoring the constant term and using binary rule $x_{i} \in {1,0}$ we have

$ - 2\sum _ { i : \alpha \in V _ { i } } x _ { i } = - 2\sum _ { i = j, i : \alpha \in V _ { i }, j : \alpha \in V _ { j } } x _ { i } x _ {j}$

$ ( \sum _ { i : \alpha \in V _ { i } } x _ { i } ) ^ { 2 } = \sum _ { i = j, i : \alpha \in V _ { i }, j : \alpha \in V _ { j } } x _ { i } x _ {j} + 2 \sum _ { i \neq j, i : \alpha \in V _ { i }, j : \alpha \in V _ { j } } x _ { i } x _ {j} $

Finally we get

$E_{A} = A \sum _ { \alpha = 1 } ^ { n } ( - \sum _ { i = j, i : \alpha \in V _ { i }, j : \alpha \in V _ { j } } x _ { i } x _ {j} + 2 \sum _ { i \neq j, i : \alpha \in V _ { i }, j : \alpha \in V _ { j } } x _ { i } x _ {j} )$

In [0]:
U = [1,2,3,4,5,6,7,8,9,10]
A = 1

def get_qubo(V):
    Q = np.zeros( (len(V), len(V)) )

    for i in range(len(V)):
        for j in range(len(V)):
            for k in range(len(U)):
                alpha = U[k]
                in_Vi = V[i].count(alpha) > 0 #Exist in V[i]
                in_Vj = V[j].count(alpha) > 0 #Exist in V[j]
                if i == j and in_Vi:
                    Q[i][j] += -1
                elif i < j and in_Vi and in_Vj:
                    Q[i][j] += 2

    return Q * A

Let's define the function to show result

In [0]:
def display_answer(list_x, energies = None, show_graph = False):
    print("Result x:", list_x)
    text = ""
    for i in range(len(list_x)):
        if(list_x[i]):
            text += str(V[i])
    print("Picked {} group(s): {}".format(sum(list_x), text))
    if energies is not None:
        print("Energy:", a.E[-1])
    if show_graph:
        plt.plot(a.E)
        plt.show()

Let's try it.

In [6]:
V = [ [1,2], [3,4,5,6], [7,8,9,10], [1,3,5], [10] ]
a = wq.opt()
a.qubo = get_qubo(V)
answer = a.sa()
display_answer(answer)

1.4076354503631592
Result x: [1, 1, 1, 0, 0]
Picked 3 group(s): [1, 2][3, 4, 5, 6][7, 8, 9, 10]


## Try on more complicated V

Let's add 2 more V and try,

In [8]:
V = [ [1,2], [3,4,5,6], [7,8,9,10], [1,3,5], [10], [7,9], [2,4,6,8] ]
a = wq.opt()
a.qubo = get_qubo(V)
answer = a.sa()
display_answer(answer)

1.4350087642669678
Result x: [1, 1, 1, 0, 0, 0, 0]
Picked 3 group(s): [1, 2][3, 4, 5, 6][7, 8, 9, 10]


Collectly we can get the result.

## Smallest Exact Cover

Next we also try on Smallest Exact Cover. We need one more cost function $E_{B}$.

$ E _ { B } = B \sum _ { i } x _ { i } $

Using the binary rule $x_{i} \in {1,0}$

$ E _ { B } = B \sum _ { i, j : i = j } x _ { i } x _ {j} $

And we get $E = E_{A} + E_{B}$

In [0]:
B = A / len(U) * 0.99
def get_qubo_min(Q):
    for i in range(len(V)):
        Q[i][i] += B
    return Q

### Trial

Let's first try only $ E _ {A} $, 5times.

Looking at the result we have 3 or 4 groups and answer are not usually select the minimum number of groups.

In [10]:
V = [ [1,2,3,4], [5,6,7,8], [9,10], [1,2], [3,4], [5,6], [7,8,9,10]]

for i in range(5):
    print("---{}times".format(i+1))
    a = wq.opt()
    a.qubo = get_qubo(V)
    answer = a.sa()
    display_answer(answer, a.E)

---1times
1.4670355319976807
Result x: [0, 1, 1, 1, 1, 0, 0]
Picked 4 group(s): [5, 6, 7, 8][9, 10][1, 2][3, 4]
Energy: -10.0
---2times
1.4914586544036865
Result x: [1, 1, 1, 0, 0, 0, 0]
Picked 3 group(s): [1, 2, 3, 4][5, 6, 7, 8][9, 10]
Energy: -10.0
---3times
1.454087495803833
Result x: [0, 0, 0, 1, 1, 1, 1]
Picked 4 group(s): [1, 2][3, 4][5, 6][7, 8, 9, 10]
Energy: -10.0
---4times
1.4568595886230469
Result x: [1, 0, 0, 0, 0, 1, 1]
Picked 3 group(s): [1, 2, 3, 4][5, 6][7, 8, 9, 10]
Energy: -10.0
---5times
1.4357664585113525
Result x: [0, 0, 0, 1, 1, 1, 1]
Picked 4 group(s): [1, 2][3, 4][5, 6][7, 8, 9, 10]
Energy: -10.0


### Try on new cost function.
Let's try $ E _ {A} + E_{B}$

Usually select the correct answer and sometimes failure.

In [12]:
for i in range(5):
    print("---{}times".format(i+1))
    a = wq.opt()
    a.qubo = get_qubo_min(get_qubo(V))
    answer = a.sa()
    display_answer(answer, a.E)

---1times
1.4373810291290283
Result x: [1, 0, 0, 0, 0, 1, 1]
Picked 3 group(s): [1, 2, 3, 4][5, 6][7, 8, 9, 10]
Energy: -9.703
---2times
1.467071533203125
Result x: [1, 1, 1, 0, 0, 0, 0]
Picked 3 group(s): [1, 2, 3, 4][5, 6, 7, 8][9, 10]
Energy: -9.703
---3times
1.4758150577545166
Result x: [1, 1, 1, 0, 0, 0, 0]
Picked 3 group(s): [1, 2, 3, 4][5, 6, 7, 8][9, 10]
Energy: -9.703
---4times
1.4710874557495117
Result x: [0, 0, 0, 1, 1, 1, 1]
Picked 4 group(s): [1, 2][3, 4][5, 6][7, 8, 9, 10]
Energy: -9.604
---5times
1.4783902168273926
Result x: [1, 1, 1, 0, 0, 0, 0]
Picked 3 group(s): [1, 2, 3, 4][5, 6, 7, 8][9, 10]
Energy: -9.703


### More difficult
Finally we try more difficult problem
The answer is to select {1,2}{3}{4}{5}{6}{7}{8}{9}{10}

Normally it shows the correct answer but sometimes we get the different answer.

In [14]:
V = [ [1,2], [3], [4], [5], [6], [7], [8], [9], [10], [2,3,4,5,6,7,8,9,10]]
for i in range(5):
    print("---{}times".format(i+1))
    a = wq.opt()
    a.qubo = get_qubo_min(get_qubo(V))
    answer = a.sa()
    display_answer(answer, a.E)

---1times
1.4879131317138672
Result x: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
Picked 9 group(s): [1, 2][3][4][5][6][7][8][9][10]
Energy: -9.108999999999998
---2times
1.4720873832702637
Result x: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Picked 1 group(s): [2, 3, 4, 5, 6, 7, 8, 9, 10]
Energy: -8.901
---3times
1.4679582118988037
Result x: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Picked 1 group(s): [2, 3, 4, 5, 6, 7, 8, 9, 10]
Energy: -8.901
---4times
1.4643397331237793
Result x: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
Picked 9 group(s): [1, 2][3][4][5][6][7][8][9][10]
Energy: -9.108999999999998
---5times
1.4716706275939941
Result x: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
Picked 9 group(s): [1, 2][3][4][5][6][7][8][9][10]
Energy: -9.108999999999998
